In [1]:
import nibabel as nib

img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\Dentition.nii.gz")
spacing = img.header.get_zooms()
print("Voxel spacing (x, y, z):", spacing)


Voxel spacing (x, y, z): (np.float32(0.9766), np.float32(0.9766), np.float32(1.25))


In [ ]:
#doing it manually means some mislabeled as not a constant level between upper and lower teeth
from scipy.ndimage import distance_transform_edt
import nibabel as nib
import numpy as np

dentition_img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\Dentition.nii.gz")
mandible_img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\mandible_manual.nii.gz")

dentition = dentition_img.get_fdata() > 0
mandible = mandible_img.get_fdata() > 0

# Get voxel spacing
spacing = dentition_img.header.get_zooms()[:3]  # (x, y, z)

# Compute distance map in millimeters
dist_map = distance_transform_edt(~mandible, sampling=spacing)

# Apply threshold in mm (e.g. 5 mm)
lower_mask = dentition & (dist_map < 7.5)
upper_mask = dentition & (~lower_mask)

# Save
affine = dentition_img.affine
nib.save(nib.Nifti1Image(lower_mask.astype(np.uint8), affine), r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\dentition_lower.nii.gz")
nib.save(nib.Nifti1Image(upper_mask.astype(np.uint8), affine), r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\dentition_upper.nii.gz")


In [9]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

ct_img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\CT.nii.gz")
X = ct_img.get_fdata()
X = (X - np.min(X)) / (np.max(X) - np.min(X))  # Normalize to [0, 1] (binary)

seg_paths = {
    "dentition_lower": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\dentition_lower.nii.gz",
    "dentition_upper": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\dentition_upper.nii.gz",
}

segments = {name: nib.load(path).get_fdata() for name, path in seg_paths.items()}
spacing = ct_img.header.get_zooms()

overlay_colors = {
    "dentition_lower": (0.2, 0.4, 1.0, 0.25), # blueish
    "dentition_upper": (1.0, 0.6, 0.0, 0.35), # orange

}

def show_slice(slice_index, plane, alpha=0.35, patient_id="UIDQQ0x1x1Hx7"):
    plt.figure(figsize=(6, 6))

    if plane == 'Axial':
        img_slice = np.rot90(X[:, :, slice_index])
        aspect = spacing[1] / spacing[0]
        seg_slices = {n: np.rot90(S[:, :, slice_index]) for n, S in segments.items()}

    elif plane == 'Coronal':
        img_slice = np.flipud(np.rot90(X[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]
        seg_slices = {n: np.flipud(np.rot90(S[:, slice_index, :])) for n, S in segments.items()}

    elif plane == 'Sagittal':
        img_slice = np.flipud(np.rot90(X[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]
        seg_slices = {n: np.flipud(np.rot90(S[slice_index, :, :])) for n, S in segments.items()}

    plt.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name, seg_slice in seg_slices.items():
        mask = seg_slice > 0.5
        if np.any(mask):
            rgba = list(overlay_colors[name])
            rgba[3] = alpha  
            overlay = np.zeros((*mask.shape, 4))
            overlay[mask] = rgba
            plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")


    legend_handles = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in overlay_colors
    ]
    leg=plt.legend(handles=legend_handles, loc='upper right', fontsize=8, frameon=False, labelcolor='white', title="Segments")
    leg.get_title().set_fontsize(10)
    leg.get_title().set_color('white')
    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=X.shape[2] // 2,
    min=0,
    max=X.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = X.shape[2] - 1
        slice_slider.value = X.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = X.shape[1] - 1
        slice_slider.value = X.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = X.shape[0] - 1
        slice_slider.value = X.shape[0] // 2


plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(show_slice, slice_index=slice_slider, plane=plane_dropdown, alpha=alpha_slider)


interactive(children=(IntSlider(value=122, description='Slice:', layout=Layout(width='80%'), max=244), Dropdow…

<function __main__.show_slice(slice_index, plane, alpha=0.35, patient_id='UIDQQ0x1x1Hx7')>

In [ ]:
#still mislabeled parts
import nibabel as nib
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# Paths
dentition_path = r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\Dentition.nii.gz"
out_lower = r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\dentition_lower.nii.gz"
out_upper = r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\dentition_upper.nii.gz"

# Load segmentation
img = nib.load(dentition_path)
data = img.get_fdata() > 0
affine = img.affine

# Get voxel coordinates
coords = np.argwhere(data)
centroid = coords.mean(axis=0)
centered = coords - centroid

# PCA
u, s, vh = np.linalg.svd(centered, full_matrices=False)
axes = vh
variances = s**2
vertical_axis = axes[np.argmin(variances)]
main_axes = axes[np.argsort(variances)[1:]]  # 2 horizontal PCA axes

# Project onto main 2D plane
xy = centered @ main_axes.T
z = centered @ vertical_axis

# Fit a smooth polynomial surface z = f(x, y)
poly = PolynomialFeatures(degree=2)
XY_poly = poly.fit_transform(xy)
model = LinearRegression().fit(XY_poly, z)
z_pred = model.predict(XY_poly)

# Split based on curved surface
lower_mask = z < z_pred
upper_mask = z >= z_pred

# Rebuild binary masks
lower = np.zeros_like(data, dtype=np.uint8)
upper = np.zeros_like(data, dtype=np.uint8)
lower[tuple(coords[lower_mask].T)] = 1
upper[tuple(coords[upper_mask].T)] = 1

# Save
nib.save(nib.Nifti1Image(lower, affine), out_lower)
nib.save(nib.Nifti1Image(upper, affine), out_upper)

print("Saved curved separation:")
print(f"  Lower -> {out_lower}")
print(f"  Upper -> {out_upper}")


Saved curved separation:
  Lower -> Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\dentition_lower.nii.gz
  Upper -> Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\dentition_upper.nii.gz


In [15]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

ct_img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\CT.nii.gz")
X = ct_img.get_fdata()
X = (X - np.min(X)) / (np.max(X) - np.min(X))  # Normalize to [0, 1] (binary)

seg_paths = {
    "dentition_lower": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\dentition_lower.nii.gz",
    "dentition_upper": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\dentition_upper.nii.gz",
}

segments = {name: nib.load(path).get_fdata() for name, path in seg_paths.items()}
spacing = ct_img.header.get_zooms()

overlay_colors = {
    "dentition_lower": (0.2, 0.4, 1.0, 0.25), # blueish
    "dentition_upper": (1.0, 0.6, 0.0, 0.35), # orange

}

def show_slice(slice_index, plane, alpha=0.35, patient_id="UIDQQ0x1x1Hx7"):
    plt.figure(figsize=(6, 6))

    if plane == 'Axial':
        img_slice = np.rot90(X[:, :, slice_index])
        aspect = spacing[1] / spacing[0]
        seg_slices = {n: np.rot90(S[:, :, slice_index]) for n, S in segments.items()}

    elif plane == 'Coronal':
        img_slice = np.flipud(np.rot90(X[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]
        seg_slices = {n: np.flipud(np.rot90(S[:, slice_index, :])) for n, S in segments.items()}

    elif plane == 'Sagittal':
        img_slice = np.flipud(np.rot90(X[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]
        seg_slices = {n: np.flipud(np.rot90(S[slice_index, :, :])) for n, S in segments.items()}

    plt.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name, seg_slice in seg_slices.items():
        mask = seg_slice > 0.5
        if np.any(mask):
            rgba = list(overlay_colors[name])
            rgba[3] = alpha  
            overlay = np.zeros((*mask.shape, 4))
            overlay[mask] = rgba
            plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")


    legend_handles = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in overlay_colors
    ]
    leg=plt.legend(handles=legend_handles, loc='upper right', fontsize=8, frameon=False, labelcolor='white', title="Segments")
    leg.get_title().set_fontsize(10)
    leg.get_title().set_color('white')
    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=X.shape[2] // 2,
    min=0,
    max=X.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = X.shape[2] - 1
        slice_slider.value = X.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = X.shape[1] - 1
        slice_slider.value = X.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = X.shape[0] - 1
        slice_slider.value = X.shape[0] // 2


plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(show_slice, slice_index=slice_slider, plane=plane_dropdown, alpha=alpha_slider)


interactive(children=(IntSlider(value=122, description='Slice:', layout=Layout(width='80%'), max=244), Dropdow…

<function __main__.show_slice(slice_index, plane, alpha=0.35, patient_id='UIDQQ0x1x1Hx7')>